In [38]:
import json
import os
import shutil
import sqlite3
from datetime import datetime
from time import sleep
#import query
import logzero
import requests
#import secret

from logzero import logger
from requests import get
import calendar
import regex as re
import pandas as pd

In [39]:
with open("cfg.json", "r") as jin:
    cfg = json.load(jin)

# rewrite to ensure formatting
with open("cfg.json", "w") as jout:
    json.dump(cfg, jout, indent=4)

In [40]:
conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

In [41]:
def db_populate():
    path = cfg["data_dir"]
    
    empty_returns = []
    conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
    cursor_proj = conn_proj.cursor()
    
    for root, dirs, files in os.walk(cfg["data_dir"]):
        for filename in files:
            if ".csv" in filename:
                try:
                    d_f = pd.read_csv(path + filename)
                    d_f.to_sql(filename[:-4], conn_proj, if_exists="replace", index=False)
                    conn_proj.commit
                except Exception as e: 
                    empty_returns.append(filename)
    
    """issue_ids = {}
    for eia_id in empty_returns:
        cursor_proj.execute(query.fetch_eia_proj_info, {"eia_id_prob": eia_id})
        eia_info = cursor_proj.fetchall()
        issue_ids[eia_id] = eia_info
    
    logger.info(
        "EIA query errors formatted(site: proj_cap, tbn_state, proj_year)"
        + str(issue_ids)
    )
    logger.info("\n")"""
    conn_proj.close
    return empty_returns

In [42]:
empty_returns = db_populate()

/Users/networkadministrator/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2653: UserWarning:

The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.

/Users/networkadministrator/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning:

Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.



In [43]:
query_zip = """
SELECT *

FROM program

LEFT JOIN zipcode on program.state_id == zipcode.state_id

WHERE is_entire_state == 1
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

state_df = pd.read_sql_query(query_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [44]:
state_df.shape

(1677080, 30)

In [45]:
query_city_zip = """
SELECT *

FROM program_city

LEFT JOIN program

on program_city.program_id == program.id

LEFT JOIN zipcode on program_city.city_id == zipcode.city_id

WHERE is_entire_state == 0
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

city_df = pd.read_sql_query(query_city_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [46]:
city_df.shape

(14048, 32)

In [47]:
query_county_zip = """
SELECT *

FROM program_county

LEFT JOIN program

on program_county.program_id == program.id

LEFT JOIN zipcode on program_county.county_id == zipcode.county_id

WHERE is_entire_state == 0
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

county_df = pd.read_sql_query(query_county_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [48]:
county_df.shape

(4889, 32)

In [49]:
query_utility_zip = """
SELECT *

FROM program_utility

LEFT JOIN program

on program_utility.program_id == program.id

LEFT JOIN utility_zipcode

on program_utility.utility_id == utility_zipcode.utility_id

LEFT JOIN zipcode on utility_zipcode.zipcode_id == zipcode.id

WHERE is_entire_state == 0
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

utility_df = pd.read_sql_query(query_utility_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [50]:
utility_df.shape

(434875, 34)

In [51]:
query_zip_zip = """
SELECT *

FROM program_zipcode

LEFT JOIN program

on program_zipcode.program_id == program.id

INNER JOIN zipcode on program_zipcode.zipcode_id == zipcode.id

WHERE is_entire_state == 0
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

zip_df = pd.read_sql_query(query_zip_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [52]:
zip_df.shape

(36, 32)

In [53]:
county_df = county_df.loc[:,~county_df.columns.duplicated()]
city_df = city_df.loc[:,~city_df.columns.duplicated()]
state_df = state_df.loc[:,~state_df.columns.duplicated()]
zip_df = zip_df.loc[:,~zip_df.columns.duplicated()]
utility_df = utility_df.loc[:,~utility_df.columns.duplicated()]

keep_cols = list(
    set(zip_df.columns).intersection(
    set(utility_df.columns)).intersection(
    set(county_df.columns)).intersection(
    set(city_df.columns)).intersection(
    set(state_df.columns))
)



dsire_by_zip = pd.concat([zip_df[keep_cols],
                          utility_df[keep_cols],
                          county_df[keep_cols],
                          city_df[keep_cols],
                          state_df[keep_cols]])

In [54]:
dsire_by_zip.columns

Index(['longitude', 'implementing_sector_id', 'name', 'fromSir',
       'start_date_text', 'created_ts', 'additional_technologies', 'city_id',
       'end_date', 'zipcode', 'administrator', 'county_id', 'updated_ts',
       'program_category_id', 'state_id', 'start_date', 'code',
       'program_type_id', 'websiteurl', 'published', 'summary', 'budget',
       'fundingsource', 'created_by_user_id', 'is_entire_state',
       'end_date_text', 'latitude', 'id'],
      dtype='object')

In [55]:
query_tech = """
SELECT * 

FROM program_technology

WHERE program_technology.technology_id == 7
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

tech_df = pd.read_sql_query(query_tech, conn_proj)

conn_proj.close

<function Connection.close>

In [56]:
check_list = tech_df.program_id.to_list()
dsire_by_zip.head()

,longitude,implementing_sector_id,name,fromSir,start_date_text,created_ts,additional_technologies,city_id,end_date,zipcode,...,websiteurl,published,summary,budget,fundingsource,created_by_user_id,is_entire_state,end_date_text,latitude,id
0,-92.0,3,Rochester Public Utilities - Commercial and In...,0,None,2006-08-09 00:00:00,None,17335.0,None,55906.0,...,https://www.rpu.org/rebates-programs/commercia...,1,<div>&#10;&#9;Rochester Public Utilities (RPU)...,None,None,492,0,None,44.0,1946
1,-78.0,3,Columbia Gas of Virginia - Home Savings Rebate...,0,None,2010-05-06 00:00:00,None,6019.0,None,20105.0,...,http://www.columbiagasva.com/ways-to-save/resi...,1,<p>&#10;&#9;Columbia Gas of Virginia offers re...,None,None,534,0,None,39.0,4127
2,-77.0,3,Columbia Gas of Virginia - Home Savings Rebate...,0,None,2010-05-06 00:00:00,None,6022.0,None,20108.0,...,http://www.columbiagasva.com/ways-to-save/resi...,1,<p>&#10;&#9;Columbia Gas of Virginia offers re...,None,None,534,0,None,39.0,4127
3,-77.0,3,Columbia Gas of Virginia - Home Savings Rebate...,0,None,2010-05-06 00:00:00,None,6022.0,None,20109.0,...,http://www.columbiagasva.com/ways-to-save/resi...,1,<p>&#10;&#9;Columbia Gas of Virginia offers re...,None,None,534,0,None,39.0,4127
4,-77.0,3,Columbia Gas of Virginia - Home Savings Rebate...,0,None,2010-05-06 00:00:00,None,6022.0,None,20110.0,...,http://www.columbiagasva.com/ways-to-save/resi...,1,<p>&#10;&#9;Columbia Gas of Virginia offers re...,None,None,534,0,None,39.0,4127


In [57]:
dsire_PV = dsire_by_zip[dsire_by_zip.id.isin(check_list)]
dsire_PV.start_date.unique()

array([None, '2015-01-26 20:00:00', '2010-01-01 05:00:00',
       '1997-01-26 21:59:32', '2012-10-01 04:00:00',
       '1997-12-16 05:00:00', '2015-01-26 05:00:00',
       '2007-01-01 00:00:00', '2010-08-12 04:00:00',
       '2017-01-01 05:00:00', '2003-05-01 04:00:00',
       '1999-01-26 05:00:00', '2004-10-01 00:00:00',
       '2004-06-01 04:00:00', '2004-01-01 05:00:00',
       '2015-01-26 20:05:00', '2015-09-05 00:00:00',
       '2005-10-01 00:00:00', '2006-03-01 05:00:00',
       '2005-12-01 00:00:00', '2006-01-01 05:00:00',
       '2021-02-01 05:00:00', '2006-06-01 04:00:00',
       '2006-07-01 04:00:00', '2007-01-01 05:00:00',
       '2010-10-01 00:00:00', '2006-10-01 00:00:00',
       '2008-01-25 05:00:00', '2007-09-01 04:00:00',
       '2007-10-01 04:00:00', '2008-02-14 05:00:00',
       '2013-05-29 04:00:00', '2008-01-09 00:00:00',
       '2008-07-01 04:00:00', '2008-08-08 04:00:00',
       '2011-01-01 00:00:00', '2008-12-15 00:00:00',
       '2009-10-01 04:00:00', '2009-03-0

In [58]:
import json
with open("../plotly_zips.geojson") as f:
    zips = json.load(f)

In [59]:
import pandas as pd
import plotly
import plotly.graph_objs as go
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

### Create chloropleth of policy types over time

Need to have Month and year ranging from first date to last for every zip code
This will need to tally the number of programs within the zip code existing during that month
Use start date - add 1 to that month and each month after (for policy count)
Use end date - subtract 1 from that month and each month after (for policy count)

If tracking other metrics, use different logic to aggregate, but accumulate per same logic pertaining to month

In [60]:
# def get_scatter_colors(sm, df):
#     grey = 'rgba(128,128,128,1)'
#     return ['rgba' + str(sm.to_rgba(m, bytes = True, alpha = 1)) if not np.isnan(m) else grey for m in df]


# for age in columns[3:24]:
#         age_data = df[age]
#         age_data.name = 'province'
#         # print(age_data.head())

#         df_tmp = age_data.copy()
#         df_tmp.index = df_tmp.index.map(match_dict)
#         df_tmp = df_tmp[~df_tmp.index.duplicated(keep=False)]

#         df_reindexed = df_tmp.reindex(index = provinces_names)

#         colormap = 'Blues'
#         cmin = df_reindexed.min()
#         cmax = df_reindexed.max()

#         sm = scalarmappable(colormap, cmin, cmax)
#         scatter_colors = get_scatter_colors(sm, df_reindexed)
#         colorscale = get_colorscale(sm, df_reindexed, cmin, cmax)
#         hover_text = get_hover_text(df_reindexed)

#         scatter_color_list.append(scatter_colors)

#         tickformat = ""

#         data = dict(type='scattermapbox',
#                     lat=lats,
#                     lon=lons,
#                     mode='markers',
#                     text=hover_text,
#                     marker=dict(size=10,
#                                 color=scatter_colors,
#                                 showscale = True,
#                                 cmin = df_reindexed.min(),
#                                 cmax = df_reindexed.max(),
#                                 colorscale = colorscale,
#                                 colorbar = dict(tickformat = tickformat)
#                                ),
#                     showlegend=False,
#                     hoverinfo='text'
#                     )

#         data_slider.append(data)

#     layers=([dict(sourcetype = 'geojson',
#                   source =sources[k],
#                   below="",
#                   type = 'line',    # the borders
#                   line = dict(width = 1),
#                   color = 'black',
#                   ) for k in range(n_provinces)
#               ] +

#               # fill_list
#             [dict(sourcetype = 'geojson',
#                   source =sources[k],
#                   below="water",
#                   type = 'fill',
#                   color = scatter_colors[k],
#                   opacity=0.8,
#                  ) for k in range(n_provinces)
#              ]
#             )

#     steps = []
#     for i in range(len(data_slider)):
#         step = dict(method='restyle',
#                     args=['visible', [False] * len(data_slider)],
#                     label='Age {}' .format(i))
#         step['args'][1][i] = True
#         steps.append(step)

#     sliders = [dict(active=0, steps=steps)]

#     layout = dict(title="2016 POPULATION",
#                   autosize=False,
#                   width=700,
#                   height=800,
#                   hovermode='closest',
#                   # hoverdistance = 30,

#                   mapbox=dict(accesstoken=MAPBOX_APIKEY,
#                               layers=layers,
#                               bearing=0,
#                               center=dict(
#                                         lat=35.715298,
#                                         lon=51.404343),
#                               pitch=0,
#                               zoom=4.9,
#                               style = 'light'),
#                   sliders=sliders,
#                   )


In [61]:
# visibility = []
# for i in range(len(data_slider)):
#     list = [False] * len(data_slider)
#     list[i] = True
#     visibility.append(list)


# steps = []
# for i in range(len(data_slider)):
#     step = dict(method='update',
#                 args=[{'visible': visibility[i]},
#                       {'mapbox.layers': layers[i]}],
#                 label='Age {}' .format(i),)
#     steps.append(step)

In [62]:
# def get_data_layout(df):

#     layers = []
#     for i in range(len(data_slider)):

#         scatter_colors = df[i]['marker']['color']

#         layer=([dict(sourcetype = 'geojson',
#                   source =sources[k],
#                   below="",
#                   type = 'line',
#                   line = dict(width = 1),
#                   color = 'black',
#                   ) for k in range(n_provinces)
#               ] +

#             [dict(sourcetype = 'geojson',
#                   source =sources[k],
#                   below="water",
#                   type = 'fill',
#                   color = scatter_colors[k],
#                   opacity=0.8,
#                  ) for k in range(n_provinces)]
#              )

#     layers.append(layer)

# return layers

In [63]:
all_zips = [i['id'] for i in zips['features']]

In [64]:
with open('../data/uszips.txt', 'w') as filehandle:
    filehandle.writelines("%s\n" % place for place in all_zips)

In [65]:
dsire_PV['start_date'] = dsire_PV.start_date.fillna(dsire_PV.created_ts)
diff_df = dsire_PV.dropna(subset = ['start_date', 'zipcode']).reset_index()
diff_df['zipcode'] = diff_df.zipcode.apply(lambda n: str(int(n)).zfill(5))
diff_df.head()

<ipython-input-65-3ef4f8b1a69a>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,index,longitude,implementing_sector_id,name,fromSir,start_date_text,created_ts,additional_technologies,city_id,end_date,...,websiteurl,published,summary,budget,fundingsource,created_by_user_id,is_entire_state,end_date_text,latitude,id
0,0,-105.0,3,Xcel Energy - Net Metering,0,None,2003-09-05 00:00:00,None,25521.0,None,...,http://www.xcelenergy.com/docs/corpcomm/psco_e...,0,Xcel Energy offers a net-metering program for ...,None,None,534,0,None,40.0,9
1,1,-105.0,3,Xcel Energy - Net Metering,0,None,2003-09-05 00:00:00,None,25521.0,None,...,http://www.xcelenergy.com/docs/corpcomm/psco_e...,0,Xcel Energy offers a net-metering program for ...,None,None,534,0,None,40.0,9
2,2,-105.0,3,Xcel Energy - Net Metering,0,None,2003-09-05 00:00:00,None,25521.0,None,...,http://www.xcelenergy.com/docs/corpcomm/psco_e...,0,Xcel Energy offers a net-metering program for ...,None,None,534,0,None,40.0,9
3,3,-105.0,3,Xcel Energy - Net Metering,0,None,2003-09-05 00:00:00,None,25521.0,None,...,http://www.xcelenergy.com/docs/corpcomm/psco_e...,0,Xcel Energy offers a net-metering program for ...,None,None,534,0,None,40.0,9
4,4,-105.0,3,Xcel Energy - Net Metering,0,None,2003-09-05 00:00:00,None,25521.0,None,...,http://www.xcelenergy.com/docs/corpcomm/psco_e...,0,Xcel Energy offers a net-metering program for ...,None,None,534,0,None,40.0,9


In [66]:
data_zips = []

# open file and read the content in a list
with open('../data/datazips.txt', 'r') as filehandle:
    filecontents = filehandle.readlines()

    for line in filecontents:
        # remove linebreak which is the last character of the string
        current_place = line[:-1]

        # add item to the list
        data_zips.append(current_place)
        
data_zips = sorted(data_zips)

In [67]:
diff_df['install_month'] = pd.to_datetime(diff_df.start_date).dt.strftime('%Y-%m')
from pandas.tseries.offsets import MonthEnd
diff_df['install_month'] = pd.to_datetime(diff_df['install_month']) + MonthEnd(1)


diff_df['end_month'] = pd.to_datetime(diff_df.end_date).dt.strftime('%Y-%m')
diff_df['end_month'] = pd.to_datetime(diff_df['end_month']) + MonthEnd(1)

diff_df.head()

,index,longitude,implementing_sector_id,name,fromSir,start_date_text,created_ts,additional_technologies,city_id,end_date,...,summary,budget,fundingsource,created_by_user_id,is_entire_state,end_date_text,latitude,id,install_month,end_month
0,0,-105.0,3,Xcel Energy - Net Metering,0,None,2003-09-05 00:00:00,None,25521.0,None,...,Xcel Energy offers a net-metering program for ...,None,None,534,0,None,40.0,9,2003-09-30,NaT
1,1,-105.0,3,Xcel Energy - Net Metering,0,None,2003-09-05 00:00:00,None,25521.0,None,...,Xcel Energy offers a net-metering program for ...,None,None,534,0,None,40.0,9,2003-09-30,NaT
2,2,-105.0,3,Xcel Energy - Net Metering,0,None,2003-09-05 00:00:00,None,25521.0,None,...,Xcel Energy offers a net-metering program for ...,None,None,534,0,None,40.0,9,2003-09-30,NaT
3,3,-105.0,3,Xcel Energy - Net Metering,0,None,2003-09-05 00:00:00,None,25521.0,None,...,Xcel Energy offers a net-metering program for ...,None,None,534,0,None,40.0,9,2003-09-30,NaT
4,4,-105.0,3,Xcel Energy - Net Metering,0,None,2003-09-05 00:00:00,None,25521.0,None,...,Xcel Energy offers a net-metering program for ...,None,None,534,0,None,40.0,9,2003-09-30,NaT


In [68]:
program_types = diff_df.program_type_id.unique()

In [69]:
program_dict = {}
for p_id in program_types:
    df = diff_df[diff_df.program_type_id == p_id]
    program_dict[p_id] = dict(zip(df.zipcode,df.start_date))
    
import pickle

with open('../data/program_dates.p', 'wb') as fp:
    pickle.dump(program_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [72]:

# policy_dict = {}
# for zc in diff_df.zipcode.unique():
#     df = diff_df[diff_df.zipcode == zc]
#     for p_id in df.program_type_id.unique():
#         sub = df[df.program_type_id == p_id]
#         policy_dict[zc] = {p_id: (list(sub.start_date)[0], list(sub.end_date)[0])}
        


In [73]:
#min_yr = diff_df.install_month.min().year
#min_mo = diff_df.install_month.min().month
min_yr = pd.to_datetime('1998-04-30 00:00:00').year
min_mo = pd.to_datetime('1998-04-30 00:00:00').month

diff_df['num_months'] = diff_df.install_month.apply(
    lambda d: (d.year - min_yr) * 12 + (d.month - min_mo)
)

diff_df['num_end_months'] = diff_df.end_month.apply(
    lambda d: (d.year - min_yr) * 12 + (d.month - min_mo)
)

diff_df_mo = diff_df[diff_df.num_months > 0]

In [74]:
#attempt to re-run above as dict comprehension to save time
diff_df_mo['s_e'] = diff_df_mo.apply(lambda x: (x.num_months, x.num_end_months), axis=1)
date_df = diff_df_mo.groupby(['zipcode','program_type_id']).s_e.apply(list).reset_index()

date_df.head()

<ipython-input-74-67f629269127>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,zipcode,program_type_id,s_e
0,00501,13,"[(186, 189.0)]"
1,00501,18,"[(201, nan)]"
2,00501,31,"[(21, nan), (201, nan)]"
3,00501,37,"[(21, nan)]"
4,00501,38,"[(77, 214.0), (220, 392.0)]"


In [75]:
policy_dict = {}
for zc in date_df.zipcode.unique():
    df = date_df[date_df.zipcode == zc]
    policy_dict[zc] = dict(zip(df.program_type_id, df.s_e))

KeyboardInterrupt: 

In [ ]:
policy_dict['91708']

In [ ]:
diff_df_mo[(diff_df_mo.zipcode == '78241') & (diff_df_mo.program_type_id == 37)]

In [ ]:
nm_df = date_df[date_df.program_type_id == 37]

nm_df['earliest_instance'] = nm_df.s_e.apply(lambda s: min([x[0] for x in s]))

nm_df.sort_values(by = 'earliest_instance', ascending = False)

In [ ]:
nm_df['earliest_instance'] = nm_df['earliest_instance'].astype(float)

In [ ]:
nm_df['zipcode'] = nm_df['zipcode'].astype('string')
nm_df.head()

In [ ]:
earliest_dict = {}
for p_id in program_types:
    p_df = date_df[date_df.program_type_id == p_id]
    p_df['earliest_instance'] = p_df.s_e.apply(lambda s: min([x[0] for x in s]))
    p_df['earliest_instance'] = p_df['earliest_instance'].astype(float)
    p_df['zipcode'] = p_df['zipcode'].astype('string')
    
    earliest_dict[p_id] = dict(zip(p_df.zipcode,p_df.earliest_instance))
    
import pickle

with open('../data/earliest_dates.p', 'wb') as fp:
    pickle.dump(earliest_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import json
import plotly.express as px


with open("../plotly_zips.geojson") as f:
    zips = json.load(f)

fig = px.choropleth_mapbox(nm_df, geojson=zips, locations='zipcode', color='earliest_instance',
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5
                          )


fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
zips

In [77]:
diff_df_mo.to_pickle("../data/diff_df_mo.pkl")

In [ ]:
#split out db creation into new notebook


#create dependencies for database
#filter out territories and zips w/out install data


#output new db with similarity connections for zip codes
#for a given policy and zip code, include dates from prior policy implementation until next implementation
## may need to filter out certain policy types

#is it worthwhile to create a regression including policy columns?

#establish zip codes of interest in cfg file
#94025 - look at policy implementations prior to 2009

#identify most similar by state